In [1]:
# Import useful libraries
import time
import config
import numpy as np
import pandas as pd
import os
import requests
import tweepy
import json

In [2]:
# Read the twitter archive data provided
wrd_archive = pd.read_csv('./twitter-archive-enhanced.csv')
wrd_archive.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [3]:
# Programmatically download the image predictions
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file_name = url.split('/')[-1]
response = requests.get(url)

start = time.time()
with open(file_name, 'wb') as f:
    f.write(response.content)
    
print('Process completed in {} seconds'.format(time.time()-start))

Process completed in 0.003014802932739258 seconds


In [4]:
# Read in the image predictions
img_predictions = pd.read_csv('./image-predictions.tsv', sep='\t')
img_predictions.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [5]:
# Create an API object to gater twitter data
consumer_key = config.API_KEY
consumer_secret = config.API_KEY_SECRET
access_token = config.ACCESS_TOKEN
access_secret = config.ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit =True,
                wait_on_rate_limit_notify=True)

In [6]:
test = api.get_status('666020888022790149', tweet_mode='extended')

In [7]:
# Pull tweet information using the ids in wrd_archive
# Extract the tweet ids from the wrd dataframe
tweet_ids = wrd_archive['tweet_id']
counter = 0
failed_attempts = {}
print('\033[1m'+'COMMENCING TWEET EXTRACTION PROCESS'+'\033[0m'+'\n')
start_time = time.time()

# Loop over each tweet id and collect the information
with open('tweet_info.json', 'w') as file:
    for tweet_id in tweet_ids:
        counter+=1
        print('Pulling...', counter, ':', tweet_id)
        print('\033[94m'+'\033[1m'+'SUCCESS'+'\033[0m')
        try:
            tweet_info = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet_info._json, file)
            file.write('\n')
        except Exception as e:
            print('\033[91m'+'\033[1m'+'Failed:'+'\033[0m', 'An Error occurred while pulling tweet informaton')
            failed_attempts[tweet_id] = e
            pass

# Print feedback on execution process  
duration = time.time() - start_time
failed = len(failed_attempts.keys())
print('\033[1m'+'\nDISPLAYING RUNTIME SUMMARY'+'\033[0m')
print('The entire process took: {} seconds'.format(round(duration, 2)))

if (failed > 0):
    print('\033[91m'+'\033[1m'+'Could not pull information for '+ str(failed) + ' tweet ids:'+'\033[0m')
    print(pd.Series(failed_attempts))
else:
    print('\033[94m'+'\033[1m'+'No failed attempts'+'\033[0m')

COMMENCING TWEET EXTRACTION PROCESS

Pulling... 1 : 892420643555336193
SUCCESS
Pulling... 2 : 892177421306343426
SUCCESS
Pulling... 3 : 891815181378084864
SUCCESS
Pulling... 4 : 891689557279858688
SUCCESS
Pulling... 5 : 891327558926688256
SUCCESS
Pulling... 6 : 891087950875897856
SUCCESS
Pulling... 7 : 890971913173991426
SUCCESS
Pulling... 8 : 890729181411237888
SUCCESS
Pulling... 9 : 890609185150312448
SUCCESS
Pulling... 10 : 890240255349198849
SUCCESS
Pulling... 11 : 890006608113172480
SUCCESS
Pulling... 12 : 889880896479866881
SUCCESS
Pulling... 13 : 889665388333682689
SUCCESS
Pulling... 14 : 889638837579907072
SUCCESS
Pulling... 15 : 889531135344209921
SUCCESS
Pulling... 16 : 889278841981685760
SUCCESS
Pulling... 17 : 888917238123831296
SUCCESS
Pulling... 18 : 888804989199671297
SUCCESS
Pulling... 19 : 888554962724278272
SUCCESS
Pulling... 20 : 888202515573088257
SUCCESS
Failed: An Error occurred while pulling tweet informaton
Pulling... 21 : 888078434458587136
SUCCESS
Pulling... 2

Pulling... 139 : 865718153858494464
SUCCESS
Pulling... 140 : 865359393868664832
SUCCESS
Pulling... 141 : 865006731092295680
SUCCESS
Pulling... 142 : 864873206498414592
SUCCESS
Pulling... 143 : 864279568663928832
SUCCESS
Pulling... 144 : 864197398364647424
SUCCESS
Pulling... 145 : 863907417377173506
SUCCESS
Pulling... 146 : 863553081350529029
SUCCESS
Pulling... 147 : 863471782782697472
SUCCESS
Pulling... 148 : 863432100342583297
SUCCESS
Pulling... 149 : 863427515083354112
SUCCESS
Pulling... 150 : 863079547188785154
SUCCESS
Pulling... 151 : 863062471531167744
SUCCESS
Pulling... 152 : 862831371563274240
SUCCESS
Pulling... 153 : 862722525377298433
SUCCESS
Pulling... 154 : 862457590147678208
SUCCESS
Pulling... 155 : 862096992088072192
SUCCESS
Pulling... 156 : 861769973181624320
SUCCESS
Failed: An Error occurred while pulling tweet informaton
Pulling... 157 : 861383897657036800
SUCCESS
Pulling... 158 : 861288531465048066
SUCCESS
Pulling... 159 : 861005113778896900
SUCCESS
Pulling... 160 : 86

Pulling... 273 : 840761248237133825
SUCCESS
Pulling... 274 : 840728873075638272
SUCCESS
Pulling... 275 : 840698636975636481
SUCCESS
Pulling... 276 : 840696689258311684
SUCCESS
Pulling... 277 : 840632337062862849
SUCCESS
Pulling... 278 : 840370681858686976
SUCCESS
Pulling... 279 : 840268004936019968
SUCCESS
Pulling... 280 : 839990271299457024
SUCCESS
Pulling... 281 : 839549326359670784
SUCCESS
Pulling... 282 : 839290600511926273
SUCCESS
Pulling... 283 : 839239871831150596
SUCCESS
Pulling... 284 : 838952994649550848
SUCCESS
Pulling... 285 : 838921590096166913
SUCCESS
Pulling... 286 : 838916489579200512
SUCCESS
Pulling... 287 : 838831947270979586
SUCCESS
Pulling... 288 : 838561493054533637
SUCCESS
Pulling... 289 : 838476387338051585
SUCCESS
Pulling... 290 : 838201503651401729
SUCCESS
Pulling... 291 : 838150277551247360
SUCCESS
Pulling... 292 : 838085839343206401
SUCCESS
Pulling... 293 : 838083903487373313
SUCCESS
Pulling... 294 : 837820167694528512
SUCCESS
Pulling... 295 : 837482249356513

Pulling... 412 : 823269594223824897
SUCCESS
Pulling... 413 : 822975315408461824
SUCCESS
Pulling... 414 : 822872901745569793
SUCCESS
Pulling... 415 : 822859134160621569
SUCCESS
Pulling... 416 : 822647212903690241
SUCCESS
Pulling... 417 : 822610361945911296
SUCCESS
Pulling... 418 : 822489057087389700
SUCCESS
Pulling... 419 : 822462944365645825
SUCCESS
Pulling... 420 : 822244816520155136
SUCCESS
Pulling... 421 : 822163064745328640
SUCCESS
Pulling... 422 : 821886076407029760
SUCCESS
Pulling... 423 : 821813639212650496
SUCCESS
Pulling... 424 : 821765923262631936
SUCCESS
Pulling... 425 : 821522889702862852
SUCCESS
Pulling... 426 : 821421320206483457
SUCCESS
Pulling... 427 : 821407182352777218
SUCCESS
Pulling... 428 : 821153421864615936
SUCCESS
Pulling... 429 : 821149554670182400
SUCCESS
Pulling... 430 : 821107785811234820
SUCCESS
Pulling... 431 : 821044531881721856
SUCCESS
Pulling... 432 : 820837357901512704
SUCCESS
Pulling... 433 : 820749716845686786
SUCCESS
Pulling... 434 : 820690176645140

Pulling... 555 : 803773340896923648
SUCCESS
Pulling... 556 : 803692223237865472
SUCCESS
Pulling... 557 : 803638050916102144
SUCCESS
Pulling... 558 : 803380650405482500
SUCCESS
Pulling... 559 : 803321560782307329
SUCCESS
Pulling... 560 : 803276597545603072
SUCCESS
Pulling... 561 : 802952499103731712
SUCCESS
Pulling... 562 : 802624713319034886
SUCCESS
Pulling... 563 : 802600418706604034
SUCCESS
Pulling... 564 : 802572683846291456
SUCCESS
Pulling... 565 : 802323869084381190
SUCCESS
Pulling... 566 : 802265048156610565
SUCCESS
Pulling... 567 : 802247111496568832
SUCCESS
Failed: An Error occurred while pulling tweet informaton
Pulling... 568 : 802239329049477120
SUCCESS
Pulling... 569 : 802185808107208704
SUCCESS
Pulling... 570 : 801958328846974976
SUCCESS
Pulling... 571 : 801854953262350336
SUCCESS
Pulling... 572 : 801538201127157760
SUCCESS
Pulling... 573 : 801285448605831168
SUCCESS
Pulling... 574 : 801167903437357056
SUCCESS
Pulling... 575 : 801127390143516673
SUCCESS
Pulling... 576 : 80

...

Pulling... 2005 : 672466075045466113
SUCCESS
Pulling... 2006 : 672272411274932228
SUCCESS
Pulling... 2007 : 672267570918129665
SUCCESS
Pulling... 2008 : 672264251789176834
SUCCESS
Pulling... 2009 : 672256522047614977
SUCCESS
Pulling... 2010 : 672254177670729728
SUCCESS
Pulling... 2011 : 672248013293752320
SUCCESS
Pulling... 2012 : 672245253877968896
SUCCESS
Pulling... 2013 : 672239279297454080
SUCCESS
Pulling... 2014 : 672231046314901505
SUCCESS
Pulling... 2015 : 672222792075620352
SUCCESS
Pulling... 2016 : 672205392827572224
SUCCESS
Pulling... 2017 : 672169685991993344
SUCCESS
Pulling... 2018 : 672160042234327040
SUCCESS
Pulling... 2019 : 672139350159835138
SUCCESS
Pulling... 2020 : 672125275208069120
SUCCESS
Pulling... 2021 : 672095186491711488
SUCCESS
Pulling... 2022 : 672082170312290304
SUCCESS
Pulling... 2023 : 672068090318987265
SUCCESS
Pulling... 2024 : 671896809300709376
SUCCESS
Pulling... 2025 : 671891728106971137
SUCCESS
Pulling... 2026 : 671882082306625538
SUCCESS
Pulling...

Pulling... 2147 : 669923323644657664
SUCCESS
Pulling... 2148 : 669753178989142016
SUCCESS
Pulling... 2149 : 669749430875258880
SUCCESS
Pulling... 2150 : 669684865554620416
SUCCESS
Pulling... 2151 : 669683899023405056
SUCCESS
Pulling... 2152 : 669682095984410625
SUCCESS
Pulling... 2153 : 669680153564442624
SUCCESS
Pulling... 2154 : 669661792646373376
SUCCESS
Pulling... 2155 : 669625907762618368
SUCCESS
Pulling... 2156 : 669603084620980224
SUCCESS
Pulling... 2157 : 669597912108789760
SUCCESS
Pulling... 2158 : 669583744538451968
SUCCESS
Pulling... 2159 : 669573570759163904
SUCCESS
Pulling... 2160 : 669571471778410496
SUCCESS
Pulling... 2161 : 669567591774625800
SUCCESS
Pulling... 2162 : 669564461267722241
SUCCESS
Pulling... 2163 : 669393256313184256
SUCCESS
Pulling... 2164 : 669375718304980992
SUCCESS
Pulling... 2165 : 669371483794317312
SUCCESS
Pulling... 2166 : 669367896104181761
SUCCESS
Pulling... 2167 : 669363888236994561
SUCCESS
Pulling... 2168 : 669359674819481600
SUCCESS
Pulling...

Rate limit reached. Sleeping for: 353


Pulling... 2196 : 668872652652679168
SUCCESS
Pulling... 2197 : 668852170888998912
SUCCESS
Pulling... 2198 : 668826086256599040
SUCCESS
Pulling... 2199 : 668815180734689280
SUCCESS
Pulling... 2200 : 668779399630725120
SUCCESS
Pulling... 2201 : 668655139528511488
SUCCESS
Pulling... 2202 : 668645506898350081
SUCCESS
Pulling... 2203 : 668643542311546881
SUCCESS
Pulling... 2204 : 668641109086707712
SUCCESS
Pulling... 2205 : 668636665813057536
SUCCESS
Pulling... 2206 : 668633411083464705
SUCCESS
Pulling... 2207 : 668631377374486528
SUCCESS
Pulling... 2208 : 668627278264475648
SUCCESS
Pulling... 2209 : 668625577880875008
SUCCESS
Pulling... 2210 : 668623201287675904
SUCCESS
Pulling... 2211 : 668620235289837568
SUCCESS
Pulling... 2212 : 668614819948453888
SUCCESS
Pulling... 2213 : 668587383441514497
SUCCESS
Pulling... 2214 : 668567822092664832
SUCCESS
Pulling... 2215 : 668544745690562560
SUCCESS
Pulling... 2216 : 668542336805281792
SUCCESS
Pulling... 2217 : 668537837512433665
SUCCESS
Pulling...

Pulling... 2337 : 666273097616637952
SUCCESS
Pulling... 2338 : 666268910803644416
SUCCESS
Pulling... 2339 : 666104133288665088
SUCCESS
Pulling... 2340 : 666102155909144576
SUCCESS
Pulling... 2341 : 666099513787052032
SUCCESS
Pulling... 2342 : 666094000022159362
SUCCESS
Pulling... 2343 : 666082916733198337
SUCCESS
Pulling... 2344 : 666073100786774016
SUCCESS
Pulling... 2345 : 666071193221509120
SUCCESS
Pulling... 2346 : 666063827256086533
SUCCESS
Pulling... 2347 : 666058600524156928
SUCCESS
Pulling... 2348 : 666057090499244032
SUCCESS
Pulling... 2349 : 666055525042405380
SUCCESS
Pulling... 2350 : 666051853826850816
SUCCESS
Pulling... 2351 : 666050758794694657
SUCCESS
Pulling... 2352 : 666049248165822465
SUCCESS
Pulling... 2353 : 666044226329800704
SUCCESS
Pulling... 2354 : 666033412701032449
SUCCESS
Pulling... 2355 : 666029285002620928
SUCCESS
Pulling... 2356 : 666020888022790149
SUCCESS

DISPLAYING RUNTIME SUMMARY
The entire process took: 3095.03 seconds
Could not pull information for 